In [0]:
import os
import pickle
import time
import glob
import sys

# use google3 ad hoc import
#from colabtools import adhoc_import
#with adhoc_import.Google3SubmittedChangelist():
#  from google3.third_party.music21 import midi, converter, chord, note

from music21 import midi, converter, chord, note, stream, instrument, duration

import numpy as np
import matplotlib.pyplot as plt

from tensorflow.keras.layers import LSTM, Input, Dropout, Dense, Activation, Embedding, Concatenate, Reshape, Bidirectional
from tensorflow.keras.layers import Flatten, RepeatVector, Permute, TimeDistributed
from tensorflow.keras.layers import Multiply, Lambda, Softmax, dot, concatenate
import tensorflow.keras.backend as K 
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.layers import Layer

from tensorflow.keras.utils import to_categorical, plot_model
import tensorflow as tf

from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping

In [0]:
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


#Set Parameters and Create Folders

In [0]:
#sys.path.append('/content/gdrive/My Drive/cs230')
project_folder = "/content/gdrive/My Drive/cs230"
#data_folder = project_folder + "/data"
cello_folder = project_folder + "/data/cello"

In [0]:
section = 'compose'
run_id = '003'
music_name = 'cello'

run_folder = project_folder + '/run/{}/'.format(section)
run_folder += '_'.join([run_id, music_name])

store_folder = os.path.join(run_folder, 'store')
data_folder = os.path.join('data', music_name)

if not os.path.exists(run_folder):
    os.makedirs(run_folder)
    os.makedirs(os.path.join(run_folder, 'store'))
    os.makedirs(os.path.join(run_folder, 'output'))
    os.makedirs(os.path.join(run_folder, 'weights'))
    
#mode = 'build'    # use this the on the first run
mode = 'load'      # use this after the firts run
# data params
seq_len = 32

# model params
embed_size = 100
rnn_units = 256
use_attention = True

#Extract Notes

In [0]:
cello_folder

'/content/gdrive/My Drive/cs230/data/cello'

#Create Note and Duration Dictionaries

In [0]:
if mode == 'build':
    
    #music_list, parser = get_music_list(data_folder)
    music_list = glob.glob(os.path.join(cello_folder, "*.mid"))
    print(len(music_list), 'files in total')

    notes = []
    durations = []

    # These dictionaries contain the number half steps required to transpose music to either C major or A minor
    majors = dict([('A-', 4),('G#', 4),('A', 3),('A#', 2),('B-', 2),('B', 1),('C', 0),('C#', -1),('D-', -1),('D', -2),('D#', -3),('E-', -3),('E', -4),('6F', -5),('F#', 6),('G-', 6),('G', 5)])
    minors = dict([('G#', 1), ('A-', 1),('A', 0),('A#', -1),('B-', -1),('B', -2),('C', -3),('C#', -4),('D-', -4),('D', -5),('D#', 6),('E-', 6),('E', 5),('F', 4),('F#', 3),('G-', 3),('G', 2)])

    for i, file in enumerate(music_list):
        print(i+1, "Parsing %s" % file)
        original_score = converter.parse(file).chordify()
        
        key = original_score.analyze('key')
        #print('original key', key)
        if key.mode == "major":
            halfSteps = majors[key.tonic.name]
        elif key.mode == "minor":
            halfSteps = minors[key.tonic.name]

        score = original_score.transpose(halfSteps)
        #print('new key', score.analyze('key'))
        notes.extend(['START'] * seq_len)
        durations.extend([0]* seq_len)

        for element in score.flat:
                
            if isinstance(element, note.Note):
                if element.isRest:
                    notes.append(str(element.name))
                    #print(element.name)
                    durations.append(element.duration.quarterLength)
                else:
                    notes.append(str(element.nameWithOctave))
                    durations.append(element.duration.quarterLength)

            if isinstance(element, chord.Chord):
                notes.append('.'.join(n.nameWithOctave for n in element.pitches))
                durations.append(element.duration.quarterLength)

    with open(os.path.join(store_folder, 'notes'), 'wb') as f:
        pickle.dump(notes, f) #['G2', 'D3', 'B3', 'A3', 'B3', 'D3', 'B3', 'D3', 'G2',...]
    with open(os.path.join(store_folder, 'durations'), 'wb') as f:
        pickle.dump(durations, f) 
else:
    with open(os.path.join(store_folder, 'notes'), 'rb') as f:
        notes = pickle.load(f) #['G2', 'D3', 'B3', 'A3', 'B3', 'D3', 'B3', 'D3', 'G2',...]
    with open(os.path.join(store_folder, 'durations'), 'rb') as f:
        durations = pickle.load(f)

In [0]:
def get_distinct(elements):
    # Get all pitch names
    element_names = sorted(set(elements))
    n_elements = len(element_names)
    return (element_names, n_elements)

def create_lookups(element_names):
    # create dictionary to map notes and durations to integers
    element_to_int = {element:number for number, element in enumerate(element_names)}
    int_to_element = {number: element for number, element in enumerate(element_names)}

    return (element_to_int, int_to_element)

In [0]:
# get the distinct sets of notes and durations
note_names, n_notes = get_distinct(notes)
duration_names, n_durations = get_distinct(durations)
distincts = [note_names, n_notes, duration_names, n_durations]

with open(os.path.join(store_folder, 'distincts'), 'wb') as f:
    pickle.dump(distincts, f)

# make the lookup dictionaries for notes and dictionaries and save
note_to_int, int_to_note = create_lookups(note_names)
duration_to_int, int_to_duration = create_lookups(duration_names)
lookups = [note_to_int, int_to_note, duration_to_int, int_to_duration]

with open(os.path.join(store_folder, 'lookups'), 'wb') as f:
    pickle.dump(lookups, f)

In [0]:
notes

NameError: ignored

#Prepare Training Data

In [0]:
def prepare_sequences(notes, durations, lookups, distincts, seq_len=32):
    """ Prepare the sequences used to train the Neural Network """

    note_to_int, int_to_note, duration_to_int, int_to_duration = lookups
    note_names, n_notes, duration_names, n_durations = distincts

    notes_network_input = []
    notes_network_output = []
    durations_network_input = []
    durations_network_output = []

    # create input sequences and the corresponding outputs
    for i in range(len(notes) - seq_len):
        notes_sequence_in = notes[i:i + seq_len]
        notes_sequence_out = notes[i + seq_len]
        notes_network_input.append([note_to_int[char] for char in notes_sequence_in])
        notes_network_output.append(note_to_int[notes_sequence_out])

        durations_sequence_in = durations[i:i + seq_len]
        durations_sequence_out = durations[i + seq_len]
        durations_network_input.append([duration_to_int[char] for char in durations_sequence_in])
        durations_network_output.append(duration_to_int[durations_sequence_out])

    n_patterns = len(notes_network_input)

    # reshape the input into a format compatible with LSTM layers
    notes_network_input = np.reshape(notes_network_input, (n_patterns, seq_len))
    durations_network_input = np.reshape(durations_network_input, (n_patterns, seq_len))
    network_input = [notes_network_input, durations_network_input]
    notes_network_output = to_categorical(notes_network_output, num_classes=n_notes)
    durations_network_output = to_categorical(durations_network_output, num_classes=n_durations)
    network_output = [notes_network_output, durations_network_output]

    return (network_input, network_output)

In [0]:
network_input, network_output = prepare_sequences(notes, durations, lookups, distincts, seq_len)

In [0]:
#print(network_output[0].shape)
#print(network_output[1].shape)

In [0]:
#print('pitch input')
#print(network_input[0][5])
#print('duration input')
#print(network_input[1][5])
#print('pitch output')
#print(network_output[0][5])
#print('duration output')
#print(network_output[1][5])

# LSTM Encoder/Decoder with Attention

In [0]:
n_notes, len(network_output[0][5])

(461, 461)

In [0]:
# def create_network(n_notes, n_durations, embed_size = 100, rnn_units = 256):
#     """ encoder/decoder with attention """

#     encoder_notes_in = Input(shape = (None,))
#     encoder_durations_in = Input(shape = (None,))

#     # Can we save these embeddings while training on jazz and then reuse to generate jazz?
#     encoder_notes = Embedding(n_notes, embed_size)(encoder_notes_in)
#     encoder_duration = Embedding(n_durations, embed_size)(encoder_durations_in) 

#     encoder_input = Concatenate()([encoder_notes, encoder_duration])

#     encoder = LSTM(rnn_units, return_sequences=True, name='lstm_enc1')(encoder_input)
#     encoder_output, encoder_h, encoder_c = LSTM(rnn_units, return_state=True, name='lstm_enc2')(encoder)
#     encoder_states = [encoder_h, encoder_c]
#     print('encoder_output', encoder_output)
#     print('encoder_states', encoder_states)
  
 
#     decoder_notes = LSTM(rnn_units, return_sequences=True, name='lstm_notes_dec')(encoder_output)#, initial_state=encoder_states)
#     print('lstm_notes_dec', decoder_notes)
#     #decoder_durations = LSTM(rnn_units, return_sequences=True, name='lstm_durations_dec')(encoder_output)
#     print('lstm_durations_dec', decoder_durations)
#     #print('decoder', decoder)
#     # end decoder

#     # Attention Mechanism
#     e = Dense(1, activation='tanh')(Concatenate()([endocer_output, decoder_out]))
#     e = Reshape([-1])(e)
#     alpha = Activation('softmax')(e)
#     c = Permute([2, 1])(RepeatVector(rnn_units)(alpha))
#     c = Multiply()([encoder_output, c])
#     c = Lambda(lambda xin: K.sum(xin, axis=1), outoput_shape=(rnn_units, ))(c)
#     c = Concatenate()([decoder_out, c])
  
#     notes_out = TimeDistributed(Dense(n_notes, activation = 'softmax', name = 'pitch'))(c)
#     print('notes_out', notes_out)
#     durations_out = TimeDistributed(Dense(n_durations, activation = 'softmax', name = 'duration'))(c)
#     print('durations_out', durations_out)
#     model = Model([encoder_notes_in, encoder_durations_in], [notes_out, durations_out])
#     opti = RMSprop(lr = 0.001)
#     model.compile(loss=['categorical_crossentropy', 'categorical_crossentropy'], optimizer=opti)
#     att_model = None
#     return model

# LSTM Baseline Network

In [0]:
# def create_network(n_notes, n_durations, embed_size = 100, rnn_units = 256):
#     """ create baseline network """

#     notes_in = Input(shape = (None,))
#     durations_in = Input(shape = (None,))

#     x1 = Embedding(n_notes, embed_size)(notes_in)
#     x2 = Embedding(n_durations, embed_size)(durations_in) 

#     x = Concatenate()([x1,x2])

#     x = LSTM(rnn_units, return_sequences=True)(x)

#     c = LSTM(rnn_units)(x)
                                    
#     notes_out = Dense(n_notes, activation = 'softmax', name = 'pitch')(c)
#     durations_out = Dense(n_durations, activation = 'softmax', name = 'duration')(c)
   
#     model = Model([notes_in, durations_in], [notes_out, durations_out])

#     opti = RMSprop(lr = 0.001)
#     model.compile(loss=['categorical_crossentropy', 'categorical_crossentropy'], optimizer=opti)

#     return model

# LSTM with Attention

In [0]:
# def create_network(n_notes, n_durations, embed_size = 100, rnn_units = 256):
#     """ create network with attention """

#     notes_in = Input(shape = (None,))
#     durations_in = Input(shape = (None,))

#     x1 = Embedding(n_notes, embed_size)(notes_in)
#     x2 = Embedding(n_durations, embed_size)(durations_in) 

#     x = Concatenate()([x1,x2])
#     x = LSTM(rnn_units, return_sequences=True)(x)
#     x = LSTM(rnn_units, return_sequences=True)(x)

#     e = Dense(1, activation='tanh')(x)
#     e = Reshape([-1])(e)
#     alpha = Activation('softmax')(e)
#     alpha_repeated = Permute([2, 1])(RepeatVector(rnn_units)(alpha))

#     c = Multiply()([x, alpha_repeated])
#     c = Lambda(lambda xin: K.sum(xin, axis=1), output_shape=(rnn_units,))(c)
                                    
#     notes_out = Dense(n_notes, activation = 'softmax', name = 'pitch')(c)
#     durations_out = Dense(n_durations, activation = 'softmax', name = 'duration')(c)
   
#     model = Model([notes_in, durations_in], [notes_out, durations_out])

#     opti = RMSprop(lr = 0.001)
#     model.compile(loss=['categorical_crossentropy', 'categorical_crossentropy'], optimizer=opti)

#     return model

# Change Log
1. add changes here

# Current Development Version

In [0]:
def create_network(n_notes, n_durations, embed_size = 100, rnn_units = 256):

    notes_in = Input(shape = (None,))
    durations_in = Input(shape = (None,))

    x1 = Embedding(n_notes, embed_size)(notes_in)
    x2 = Embedding(n_durations, embed_size)(durations_in) 

    x = Concatenate()([x1,x2])
    x = Bidirectional(LSTM(rnn_units, return_sequences=True))(x)
    x = LSTM(rnn_units, return_sequences=True)(x)

    e = Dense(1, activation='tanh')(x)
    e = Reshape([-1])(e)
    alpha = Activation('softmax')(e)
    alpha_repeated = Permute([2, 1])(RepeatVector(rnn_units)(alpha))

    c = Multiply()([x, alpha_repeated])
    c = Lambda(lambda xin: K.sum(xin, axis=1), output_shape=(rnn_units,))(c)
                                    
    notes_out = Dense(n_notes, activation = 'softmax', name = 'pitch')(c)
    durations_out = Dense(n_durations, activation = 'softmax', name = 'duration')(c)
   
    model = Model([notes_in, durations_in], [notes_out, durations_out])

    opti = RMSprop(lr = 0.001)
    model.compile(loss=['categorical_crossentropy', 'categorical_crossentropy'], optimizer=opti)

    return model

In [0]:
model = create_network(n_notes, n_durations, embed_size, rnn_units)
model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, None)]       0                                            
__________________________________________________________________________________________________
input_2 (InputLayer)            [(None, None)]       0                                            
__________________________________________________________________________________________________
embedding (Embedding)           (None, None, 100)    45600       input_1[0][0]                    
__________________________________________________________________________________________________
embedding_1 (Embedding)         (None, None, 100)    1900        input_2[0][0]                    
______________________________________________________________________________________________

In [0]:
run_folder

'/content/gdrive/My Drive/cs230/run/compose/002_cello'

#Training

In [0]:
weights_folder = os.path.join(run_folder, 'weights')

In [0]:
checkpoint1 = ModelCheckpoint(
    os.path.join(weights_folder, "weights-improvement-{epoch:02d}-{loss:.4f}-bigger.h5"),
    monitor='loss',
    verbose=0,
    save_best_only=True,
    mode='min'
)

checkpoint2 = ModelCheckpoint(
    os.path.join(weights_folder, "weights.h5"),
    monitor='loss',
    verbose=0,
    save_best_only=True,
    mode='min'
)

early_stopping = EarlyStopping(
    monitor='loss'
    , restore_best_weights=True
    , patience = 10
)


callbacks_list = [
    checkpoint1
    , checkpoint2
    , early_stopping
 ]

model.save_weights(os.path.join(weights_folder, "weights.h5"))
model.fit(network_input, network_output
          , epochs=2000000, batch_size=32
          , validation_split = 0.2
          , callbacks=callbacks_list
          , shuffle=True
         )


Epoch 1/2000000
720/720 [==============================] - 14s 19ms/step - loss: 3.9687 - pitch_loss: 3.2680 - duration_loss: 0.7007 - val_loss: 4.6908 - val_pitch_loss: 3.9020 - val_duration_loss: 0.7888
Epoch 2/2000000
720/720 [==============================] - 12s 17ms/step - loss: 3.4610 - pitch_loss: 2.9139 - duration_loss: 0.5471 - val_loss: 4.8594 - val_pitch_loss: 4.0505 - val_duration_loss: 0.8089
Epoch 3/2000000
720/720 [==============================] - 12s 17ms/step - loss: 3.2768 - pitch_loss: 2.7753 - duration_loss: 0.5015 - val_loss: 4.6424 - val_pitch_loss: 3.7960 - val_duration_loss: 0.8464
Epoch 4/2000000
720/720 [==============================] - 12s 17ms/step - loss: 3.0968 - pitch_loss: 2.6289 - duration_loss: 0.4679 - val_loss: 4.8405 - val_pitch_loss: 4.0061 - val_duration_loss: 0.8344
Epoch 5/2000000
720/720 [==============================] - 12s 17ms/step - loss: 2.9126 - pitch_loss: 2.4819 - duration_loss: 0.4308 - val_loss: 4.9455 - val_pitch_loss: 4.0789 - v

# Generate the Music

In [0]:
def sample_with_temp(preds, temperature):

    if temperature == 0:
        return np.argmax(preds)
    else:
        preds = np.log(preds) / temperature
        exp_preds = np.exp(preds)
        preds = exp_preds / np.sum(exp_preds)
        return np.random.choice(len(preds), p=preds)

In [0]:
# prediction params
notes_temp=0.5
duration_temp = 0.5
max_extra_notes = 50
max_seq_len = 32
seq_len = 32

notes = ['START']
durations = [0]

if seq_len is not None:
    notes = ['START'] * (seq_len - len(notes)) + notes
    durations = [0] * (seq_len - len(durations)) + durations


sequence_length = len(notes)